In [1]:
#
# 사용자 설정
#```
USER = 'sjh'
# USER = 'hse'

# basic package
import os, re, time
import pandas as pd
import numpy as np
from sys import platform
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# for crawling
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
chrome_opt = webdriver.ChromeOptions()

# to unable image
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_opt.add_experimental_option("prefs", prefs) 

if USER == 'sjh':
    print('>> Current OS: ', platform)
    if 'darwin' in platform:
        driverPATH = '/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads/chromedriver'
        chrome_opt.add_argument('--kiosk')
    else:
        driverPATH = r'C:\Users\jhun1\Dropbox\My PC (LAPTOP-VLNR6K8R)\Downloads\chromedriver_win32\chromedriver'
        chrome_opt.add_experimental_option("excludeSwitches", ["enable-logging"])
        chrome_opt.add_argument('--start-maximized') # 창 최대
        
elif USER == 'hse':
    driverPATH = "C:\Download\chromedriver.exe"
    

class Crawler:
    def __init__(self, USER, seleniumPath, chromeOption):
        self.user = USER
        self.seleniumPath = seleniumPath
        self.chromeOption = chromeOption
        self.driver = webdriver.Chrome(executable_path=seleniumPath, chrome_options=chromeOption)
        self.acc = ActionChains(self.driver)
    
    @staticmethod
    #데이터 정리 함수
    def get_str(str_tmp):
        str_tmp = str_tmp.replace("\n", " ")
        str_tmp = str_tmp.replace("\t", " ")
        str_tmp = str_tmp.replace("                ", " ")
        str_tmp = str_tmp.strip()
        return str_tmp


    # 댓글이던, 영상이던 최하단까지 스크롤해줌
    def scroll(self, scroll_time, wait_time, testOpt = False):
        last_page_height = self.driver.execute_script(
            "return document.documentElement.scrollHeight"
        )

        stand_height = 0
        sub_height = last_page_height

        #!# 스크롤을 불필요하게 길게함
        start = time.time()
        while True:
            current_height = self.driver.execute_script("return document.documentElement.scrollHeight")

            for i in range(10):
                self.driver.execute_script(f"window.scrollTo(0, {stand_height + (sub_height/10 * i)});")
                time.sleep(scroll_time)
            time.sleep(wait_time)

            new_page_height = self.driver.execute_script("return document.documentElement.scrollHeight")
            stand_height = last_page_height
            sub_height = new_page_height - last_page_height

            if new_page_height == last_page_height:
                break

            endTime = time.time()
            howLong = endTime - start
            if testOpt == True and (howLong > 120): # 테스트모드 일 때는 1분만 스크롤함
                 break
            last_page_height = new_page_height
            time.sleep(wait_time)


    # 유튜브 들어가서 검색어 넣고 크롤링함
    def oneSearch(self, searchWord, testOpt=False):
        # 멀티프로세싱하려고 일부러 드라이버를 따로 부르는 시도 -> window, ipynb, interactive cell is impossible to multiprocessing
        # driver = webdriver.Chrome(executable_path=self.seleniumPath, chrome_options=self.chromeOption)

        output = pd.DataFrame(columns='영상명 채널명 조회수 URL 댓글수 reply'.split())
        start = time.time()
        self.driver.get(f'https://www.youtube.com/results?search_query={searchWord}')
        time.sleep(2) # 충분히 element 로드

        
        Crawler.scroll(self, 2, 0.5, testOpt=testOpt)
        
        # 스크롤을 하드하게 하면 몇 개의 박스가 나올지 모름
        renderBoxLS = self.driver.find_elements(By.CLASS_NAME, 'style-scope ytd-section-list-renderer')
        for render in renderBoxLS:
            container = self.driver.find_element(By.ID, 'contents')
            videoLS = container.find_elements(By.XPATH, '//*[@id="meta"]')

        for idx, video in enumerate(videoLS):
            try:
                title = video.find_element(By.CSS_SELECTOR, '#video-title > yt-formatted-string').text
                view = video.find_element(By.CSS_SELECTOR, '#metadata-line > span:nth-child(1)').text
                date = video.find_element(By.CSS_SELECTOR, '#metadata-line > span:nth-child(2)').text
                href = video.find_element(By.CSS_SELECTOR, '#video-title').get_attribute('href')

                output.loc[idx, '영상명'] = title
                output.loc[idx, '조회수'] = view
                output.loc[idx, 'URL'] = href
   
            except NoSuchElementException:
                break

        print('>> function performance(sec): ', time.time()-start)
        return output

    def D1(self, searchWordLS):
        output = pd.DataFrame(columns='영상명 채널명 조회수 URL 댓글수 reply'.split())

        for word in searchWordLS:
            print('\n>> Word ', word, ' crawling start')
            temp = Crawler.oneSearch(self, word, testOpt = False)
            output = output.append(temp, ignore_index=True)
            print(f'>> {word} crawled.', temp.shape)

        output.reset_index(drop=True, inplace=True)
        return output

>> Current OS:  win32


In [ ]:
crawler = Crawler(USER, driverPATH, chrome_opt)
D1 = crawler.D1(['재중동포', '조선족'])
# D1 = crawler.D1(['재중동포'])
# finalRS = crawler.D2(D1)

In [2]:
D1.to_excel('result(220808).xlsx', index=False)

NameError: name 'D1' is not defined

In [3]:
class ClientRule:
    def __init__(self, crawledDf):
        if type(crawledDf) == pd.DataFrame:
            self.df = crawledDf
        elif type(crawledDf) == str:
            self.df = pd.read_excel(os.path.join(os.getcwd(), crawledDf))

    # 조회수에 '878 views' '212k views' 이런거 다 고쳐야함
    @staticmethod
    def viewCleaner(inputView):
        delete = len('views') + 1 # 공백까지 카운트
        txt = inputView[:-delete]
        if re.search('[.KM]', txt): # . K M이 있나 확인
            if re.search('K', txt):
                kIdx = txt.find('K') 
                txt2 = txt[:kIdx]
                output = float(txt2) * 1000
            elif re.search('M', txt):
                mIdx = txt.find('M')
                txt2 = txt[:mIdx]
                output = float(txt2) * 1000000

        else: # 숫자만 있다는 것임
            try:
                output = float(txt)
            except ValueError:
                output = 0.0

        return output
    
    @staticmethod
    def duplicate(inputDf):
        output = inputDf.drop_duplicates(subset=['URL'], keep='first', ignore_index=True)
        return output

    def cleaning(self, topN = 100):
        ToDrop = []
        for idx, val in enumerate(self.df['영상명'].values):
            if ('재중동포' not in val) and ('조선족' not in val):
                ToDrop.append(idx)
        
        before = self.df.shape[0]
        sliceDf = self.df.drop(ToDrop, axis=0, inplace=True)
        after =  self.df.shape[0]
        print('>> 제목으로 정렬해서 탈락함: ', before - after)

        self.df['조회수'] = self.df['조회수'].apply(lambda x: ClientRule.viewCleaner(x))
        self.df = ClientRule.duplicate(self.df)
    

        sortedDf = self.df.sort_values(by=['조회수'])

        sliceDf = sortedDf.iloc[-topN:, :]
        sliceDf.reset_index(drop=True, inplace=True)
        display(sliceDf.head())
        return sliceDf 

In [4]:
cleaner = ClientRule('result(220808).xlsx')
del cleaner.df['Unnamed: 0']
D1_RS = cleaner.cleaning()

>> 제목으로 정렬해서 탈락함:  184


,영상명,채널명,조회수,URL,댓글수,reply
0,중국은 지금 '아리랑' 열풍 중...중국동포(조선족) 귀여운 가수 김시연의 노래! ...,NaN,5400.0,https://www.youtube.com/watch?v=RsZ0Y6xzVEg,NaN,NaN
1,"신기에 가까운 ""물동이 춤"" 연길시조선족무용단",NaN,5500.0,https://www.youtube.com/watch?v=bL1av4OGvtU,NaN,NaN
2,충격 조선족 오열 #shorts,NaN,5500.0,https://www.youtube.com/shorts/QvhUypqlhFI,NaN,NaN
3,한송이씨는 우주로 가라 연변조선족자치주 만세 세계적 웃음거리로다,NaN,5500.0,https://www.youtube.com/shorts/e9zjH8sUc4E,NaN,NaN
4,동계올림픽에 조선족분이 한복 입은것에 대해... 라이브중 질문대답!,NaN,5600.0,https://www.youtube.com/watch?v=nxCFVuiJIWw,NaN,NaN


In [12]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


class NextCrawler(Crawler):
    def __init(self, USER, seleniumPath, chromeOption):
        super().__init__(USER, seleniumPath, chromeOption)    
        self.driver = webdriver.Chrome(executable_path=seleniumPath, chrome_options=chromeOption)
        self.acc = ActionChains(self.driver)
        

        # 댓글이던, 영상이던 최하단까지 스크롤해줌
    def scroll(self, waitTime, currentURL, testOpt = False):
        self.acc.send_keys(Keys.END).perform() # 처음에는 한번 내려줌
        fullLoaded = 0
        totalScrollCount = 0
        
        try:
            commentExist = self.driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer/div[3]/ytd-message-renderer/yt-formatted-string[1]/span')
            if commentExist.text == 'Comments are turned off.':
                print('>> comments are turned off: ', currentURL)
                return []
        
        except NoSuchElementException:
            while fullLoaded < 3: # 댓글 수 변화가 없을 때까지 로드함
                replyBox = self.driver.find_element(By.XPATH, '//*[@id="contents"]') # 댓글 박스 전체
                
                replyCount_before = replyBox.find_elements(By.XPATH, '//*[@id="body"]') # 각 댓글


                self.acc.send_keys(Keys.PAGE_DOWN).perform()  # 키보드 END 키를 눌러서 동영상 더 로드하기 -> 절대 end를 누르면 안되고 page_down을 눌러야함
                totalScrollCount += 1 # 무턱대고 오래 스크롤할 때가 있어서, 이렇게 에러코드 넣어줌
                if totalScrollCount > 100:
                    break
                time.sleep(waitTime)

                replyCount_after = replyBox.find_elements(By.XPATH, '//*[@id="body"]')

                if len(replyCount_before) == len(replyCount_after):
                    fullLoaded += 1
                else:
                    fullLoaded = 0 # 이걸 해줘야, 데이터 로드에 잠깐 시간이 걸려도 불필요하게 카운트되지 않음
            if testOpt == True:
                print('>> all scrolled', len(replyCount_after))
            return replyCount_after
    
    # @staticmethod
    # def shortsReplcae(inputDf):
    #     LS = list(inputDf['URL'].values)
    #     newLS = []
        
    #     for val in LS:
    #         if 'short' in val:
    #             newVal = val.replace('shorts', 'watch?v=')
    #             newLS.append(newVal)
    #         else:
    #             newLS.append(newVal)

    #     inputDf['URL'] = newLS
    #     return inputDf
        
    
    # 한 영상의 댓글을 긁어옴
    def earnReply(self, href):
        output = pd.DataFrame(columns='영상명 채널명 조회수 URL reply'.split())
        self.driver.get(href)
        wait = WebDriverWait(self.driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]')))
        

        replyCount_after = NextCrawler.scroll(self, 1.5, href) # 1초하니까 댓글이 있는데도 안긁히는 영상이 생김
        if len(replyCount_after) == 0 :
            print('>> no reply: ', href)

        if self.user == 'hse':
            html_source = self.driver.page_source
            soup = BeautifulSoup(html_source, 'lxml')

            youtube_user_IDs = soup.select('div#header-author > a > span')
            youtube_comments = soup.select('yt-formatted-string#content-text')

            str_comments = []
            for i in range(len(youtube_comments)):
                str_comments.append(Crawler.get_str(str(youtube_comments[i].text)))
        
        elif self.user == 'sjh':
            str_comments = []

            for oneReply in replyCount_after:
                reply = oneReply.find_element(By.CSS_SELECTOR, '#content-text').text
                str_comments.append(reply)
        
        title = self.driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/div[5]/div[1]/div[2]/ytd-video-primary-info-renderer/div/h1/yt-formatted-string').text
        view = self.driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/div[5]/div[1]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/ytd-video-view-count-renderer/span[1]').text
        channel = self.driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/div[5]/div[2]/div[2]/ytd-video-secondary-info-renderer/div/div/ytd-video-owner-renderer/div[1]/ytd-channel-name/div/div/yt-formatted-string/a').text

        output['reply'] = str_comments

        # 댓글을 먼저 채우고 밑에를 채워야함
        output['영상명'] = title
        output['채널명'] = channel
        output['조회수'] = view
        output['URL'] = href
        return output
    
    @staticmethod
    def shortCounter(inputDf):
        LS = list(inputDf['URL'].values)
        count = 0

        for val in LS:
            if 'short' in val:
                count += 1
        print('>> 쇼츠 개수: ', count)
        # return count

    def D2(self, resultOfD1,
                fromIdx=0, # 몇번째 for loop부터 돌릴지..
                howMuch=5, # 크롤러가 오류가 날 수 있으므로, N등분해서 크롤링해줘야함
                manualMode = False, # 수동으로 인덱싱할 때 
                ):
        output = pd.DataFrame(columns='영상명 채널명 조회수 URL reply'.split())
        resultOfD1.reset_index(drop=True, inplace=True)
        NextCrawler.shortCounter(resultOfD1) # find shorts video count 
        window = round(resultOfD1.shape[0]/howMuch)

        url = 0 # 에러핸들링용 변수설정임, 아무 의미 없음
        try:
            for N in range(fromIdx, howMuch):
                if N == (howMuch - 1): # last index
                    sliceDf = resultOfD1.loc[N*window : , :]
                else:
                    sliceDf = resultOfD1.loc[N*window : (N+1)*window, :]
                
                sliceDf.reset_index(drop=True, inplace=True) # 해줘야 아래 코드가 돌아감
                
                if manualMode == True:
                    for idx in tqdm(range(20), desc=f'\n>> Reply crawling, {resultOfD1.shape[0]}'):
                        url = resultOfD1.loc[idx, 'URL']
                        if 'short' in url:
                            newUrl = url.replace('shorts/', 'watch?v=')
                            currentOutput = NextCrawler.earnReply(self, newUrl)
                        else:
                            currentOutput = NextCrawler.earnReply(self, url)
                        print(f'>> {currentOutput.shape[0]} replies collected')
                        output = output.append(currentOutput, ignore_index=True)
            
                else:
                    for idx in tqdm(range(sliceDf.shape[0]), desc=f'\n>> Reply crawling, {sliceDf.shape[0]}'):
                        url = sliceDf.loc[idx, 'URL']
                        if 'short' in url:
                            newUrl = url.replace('shorts/', 'watch?v=')
                            currentOutput = NextCrawler.earnReply(self, newUrl)
                        else:
                            currentOutput = NextCrawler.earnReply(self, url)
                        print(f'>> {currentOutput.shape[0]} replies collected')
                        output = output.append(currentOutput, ignore_index=True)

            
            return output
        
        except:
            print('>> 여기 긁다가 멈춤: ', url)
            return output
        

In [ ]:
# nxtCrawler = NextCrawler(USER, driverPATH, chrome_opt)
# nxtCrawler.earnReply('https://www.youtube.com/watch?v=tz0iEoi-OSo')

In [13]:
## 전체 크롤러
# nxtCrawler = NextCrawler(USER, driverPATH, chrome_opt)
# D2_RS = nxtCrawler.D2(resultOfD1= D1_RS,
#                     # fromIdx=1``
#                     )


## 22.08.09 쇼츠만
ToDrop = []
for idx, val in enumerate(D1_RS['URL'].values):
    if 'short' not in val:
        ToDrop.append(idx)

D1_RS_only_short = D1_RS.drop(ToDrop, axis=0)


nxtCrawler = NextCrawler(USER, driverPATH, chrome_opt)
D2_RS = nxtCrawler.D2(resultOfD1= D1_RS_only_short,
                    howMuch = 2,
                    # fromIdx=1``
                    )

>> 쇼츠 개수:  22



>> Reply crawling, 12:   8%|▊         | 1/12 [00:10<01:56, 10.64s/it]

>> 20 replies collected



>> Reply crawling, 12:  17%|█▋        | 2/12 [00:20<01:41, 10.18s/it]

>> 2 replies collected


...

>> 20 replies collected



>> Reply crawling, 11:  91%|█████████ | 10/11 [01:27<00:09,  9.20s/it]

>> 20 replies collected



>> Reply crawling, 11: 100%|██████████| 11/11 [01:37<00:00,  8.83s/it]

>> 20 replies collected


In [14]:
D2_RS.to_excel('오직 쇼츠만.xlsx', index=False)

In [ ]:
D2_RS = pd.read_excel('crawled(220807)v2.xlsx')

class FinalClenaer:
    def __init__(self, D1, D2):
        self.d1 = D1
        self.d2 = D2

    
    def commentAreTurnedOff(self, inputDF): # 최종 결과를 넣어야함
        fullUrl = list(np.unique(self.d1['URL'].values))
        assert len(fullUrl) == 100

        withoutShort = [url for url in fullUrl if 'short' not in url]
        print(len(withoutShort))
        crawled = list(np.unique(inputDF['URL'].values))
        
        shouldBeChecked = []
        for url in withoutShort:
            if url not in crawled:
                shouldBeChecked.append(url)

        print('>> should be checked: ', shouldBeChecked)
        return shouldBeChecked

    def customCleaner(self):
        assert self.d1.shape[0] == 100, print('>> ', self.d1.shape[0])
        self.d2.reset_index(drop=True, inplace=True)
        before = self.d2.shape[0]
        filledIdx = [idx for idx, title in enumerate(self.d2['영상명'].values) if type(title) == str]
        forMerge = self.d2.loc[filledIdx, :]
        print('>> for merge: ', forMerge.shape[0])

        notFilled = self.d2.drop(filledIdx, axis=0)
        print('>> to be filled: ', notFilled.shape[0])
        assert notFilled.shape[0] + forMerge.shape[0] == before # 이걸 꼭 해줘야함
        
        crawled = list(np.unique(notFilled['URL'].values))
        print('>> to be filled video: ', len(crawled))
        for idx, (url, title, view) in enumerate(zip(self.d1['URL'], self.d1['영상명'], self.d1['조회수'])): # 전체 영상에 대해서 루프돌려서
            if url in crawled:
                # if idx < 15:
                #     whereIsit = notFilled[notFilled['URL'] == url].index
                #     notFilled.loc[whereIsit, '영상명'] = title
                #     notFilled.loc[whereIsit, '조회수'] = view
                    # print(notFilled.loc[whereIsit, :])
                whereIsit = notFilled[notFilled['URL'] == url].index
                notFilled.loc[whereIsit, '영상명'] = title
                notFilled.loc[whereIsit, '조회수'] = view

        output = pd.concat([forMerge, notFilled], axis=0)
        return output

cleaner = FinalClenaer(D1= D1_RS, D2= D2_RS)
finalRS = cleaner.customCleaner() # 데이터 정리
NextCrawler.shortCounter(D1_RS) # 쇼츠 개수 파악
beChecked = cleaner.commentAreTurnedOff(inputDF = finalRS) # 댓글 없는 영상 재검토


with pd.ExcelWriter('finalRS(220808).xlsx') as writer:
    D1_RS.to_excel(writer, sheet_name='선별된 영상 100개')
    finalRS.to_excel(writer, sheet_name='댓글 크롤링 결과')

In [ ]:
## 08.09 쇼츠랑 합치기
